In [1]:
import os

from elasticsearch import Elasticsearch
import json
from openai import OpenAI
from tqdm.auto import tqdm

In [2]:
es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': '2c4e9162811c', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'MpbbeFB4RVKexejDUgyM1w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
INDEX_NAME = "permit-questions"
PERMIT_TYPE = "permit_type"

In [7]:
with open('permit_data.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [8]:
docs_raw

[{'permit_type': 'solar-pv-installation',
  'documents': [{'text': 'Yes, most jurisdictions require a permit to install PV systems to ensure safety and compliance.',
    'type': 'Permit Requirements',
    'question': 'Do I need a permit to install PV?'},
   {'text': 'You typically need an electrical and building permit for PV installations. Check with your local building authority.',
    'type': 'Permit Requirements',
    'question': 'Which permit should I get if I want to install PV?'},
   {'text': 'Some areas allow DIY installations, but most require a licensed electrician for safety compliance.',
    'type': 'Permit Requirements',
    'question': 'Can I install PV myself, or do I need a licensed contractor?'},
   {'text': 'Yes, either you or your installer will need to obtain a permit before installation.',
    'type': 'Permit Requirements',
    'question': 'If I hire someone to install PV, do I need a permit?'},
   {'text': 'Some HOAs have additional requirements, but state and loc

In [14]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [10]:
documents = []

for item in docs_raw:
    for doc in item['documents']:
        doc[PERMIT_TYPE] = item[PERMIT_TYPE]
        documents.append(doc)

In [11]:
documents[0]

{'text': 'Yes, most jurisdictions require a permit to install PV systems to ensure safety and compliance.',
 'type': 'Permit Requirements',
 'question': 'Do I need a permit to install PV?',
 'permit_type': 'solar-pv-installation'}

In [14]:
query = 'Can I install PV without a permit?'

In [13]:
query_pv = "How long does it take to get a permit for PV"

In [15]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "permit_type": PERMIT_TYPE
                    }
                }
            }
        }
    }
    response = es_client.search(index=INDEX_NAME, body=search_query)

    results_docs = []

    for hit in response['hits']['hits']:
        results_docs.append(hit['_source'])
    return results_docs

In [16]:
elastic_search(query)

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [permit-questions]', permit-questions, index_or_alias)

In [13]:
def rag(query):
    search_results = elastic_search(query)
    promt=building_promt(query, search_results)
    answer=llm(prompt)
    return answer

In [17]:
rag(query)

NameError: name 'building_promt' is not defined